# Random Walk Sampling

In this notebook, we will compute the Random Walk ffrom a set of seeds using cuGraph.  We will then try the same approach using NetworkX and illustrate how much faster and easier it is with cuGraph.


| Author Credit |    Date    |  Update      | cuGraph Version |  Test Hardware |
| --------------|------------|--------------|-----------------|----------------|
| Brad Rees     | 04/20/2021 | created      | 0.19            | GV100, CUDA 11.0


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)


Because the test data has vertex IDs starting at 1, the auto-renumber feature of cuGraph (mentioned above) will be used so the starting vertex ID is zero for maximum efficiency. The resulting data will then be auto-unrenumbered, making the entire renumbering process transparent to users.

In [1]:
#  Import the modules
import cugraph
import cudf

In [2]:
# Read The Data
# Define the path to the test data  
datafile='../data/karate-data.csv'

gdf = cudf.read_csv(datafile, delimiter='\t', names=['src', 'dst'], dtype=['int32', 'int32'] )

In [3]:
gdf['wt'] = 1.0

In [4]:
# Create a Graph - using the source (src) and destination (dst) vertex pairs from the Dataframe 
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src', destination='dst', edge_attr='wt')

In [5]:
# some stats on the graph
(G.number_of_nodes(), G.number_of_edges() )

(34, 78)

In [6]:
# create a list with the seeds
seeds = [17,19]

In [7]:
rw, os = cugraph.random_walks(G, seeds, 4)

the function return two results, let's look at the second one first.  The seed offset, which is a cuDF Series,
is orders based on the order of the seeds and provides an offset into the results dataframe
of where the path for that seed starts.  The series is one larger than the number of seeds, so it is easy to compute the number of rows that make up the path.  We need to know the number of rows since that could be different than the max path length.

In [8]:
os

0    0
1    3
2    6
dtype: int64

In [9]:
for i in range(len(seeds)):
    print(f"seed {seeds[i]} starts at index {os[i]} and is {os[1 + 1] - os[1]} rows")

seed 17 starts at index 0 and is 3 rows
seed 19 starts at index 3 and is 3 rows


Currently Network does not have a random walk function.  There is code on StackOverflow that generats a random walk by getting a vertice and then randomly selection a neighbor and then repeating the process.  

-----
Copyright (c) 2021, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
